In [1]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

df = pd.read_csv("/home/beamx/Downloads/email.csv")

In [2]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5573 entries, 0 to 5572
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5573 non-null   object
 1   Message   5573 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [6]:
df.isna().sum()

Category    0
Message     0
dtype: int64

In [7]:
# Step 2: Data Cleaning & Preprocessing
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove email addresses
    text = re.sub(r'\S*@\S*\s?', '', text)
    # Remove URLs
    text = re.sub(r'http[s]?://\S+', '', text)
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove special characters, numbers, and punctuation
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['cleaned_message'] = df['Message'].apply(preprocess_text)

In [8]:
# Step 3: Split Data into Training and Test Sets
X = df['cleaned_message']
y = df['Category'].apply(lambda x: 1 if x == "spam" else 0)  # Convert labels to binary (1 = spam, 0 = ham)

In [9]:
df.head()

,Category,Message,cleaned_message
0,ham,"Go until jurong point, crazy.. Available only ...",go until jurong point crazy available only in ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in a wkly comp to win fa cup final ...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor u c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah i dont think he goes to usf he lives aroun...


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.feature_extraction.text import TfidfVectorizer

# Convert the text data into numerical features again with n-grams (1 to 2-grams)
tfidf = TfidfVectorizer(stop_words='english', max_features=2000, ngram_range=(1, 2))
X_train_vec = tfidf.fit_transform(X_train) # Fit and transform the training data
X_test_vec = tfidf.transform(X_test) # Transform the testing data based on the fitted model

# Check the shape of the transformed training data and the length of the training labels
print(X_train_vec.shape)
print(len(y_train))


(4458, 2000)
4458


In [12]:
X_train, y_train = X_train.align(y_train, join='inner')

In [20]:
model = LogisticRegression(solver='liblinear', max_iter=1000)
model.fit(X_train_vec, y_train)

LogisticRegression(max_iter=1000, solver='liblinear')

In [21]:
#Evaluate the Model
y_pred = model.predict(X_test_vec)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.968609865470852

Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98       958
           1       0.99      0.78      0.88       157

    accuracy                           0.97      1115
   macro avg       0.98      0.89      0.93      1115
weighted avg       0.97      0.97      0.97      1115


Confusion Matrix:
 [[957   1]
 [ 34 123]]


In [29]:
def predict_spam(text):
    cleaned_text = preprocess_text(text)  # Preprocess the input text
    text_vec = tfidf.transform([cleaned_text])  # Convert text to TF-IDF vector
    prediction = model.predict(text_vec)  # Predict using the model
    return "Spam" if prediction[0] == 1 else "Not Spam"

# Test with a sample email
sample_email = "Welcome to ATSFriendly.com!"
print("\nPrediction for sample email:", predict_spam(sample_email))


Prediction for sample email: Not Spam
